## Hackathon V2

## Setting up Environment:


In [34]:
import pandas as pd
import numpy as np
import os
from termcolor import colored
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold,train_test_split
from catboost import CatBoostClassifier
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import math
import datetime
import warnings
warnings.filterwarnings('ignore')

## Data Cleaning & Feature Creation Functions:


In [35]:
def df_null_percentage(df = None):
    """
    input: DataFrame
    returns: % of Nulls in the dataframe's rows 
    """
    train_missing = (1 - df.count()/len(df)) * 100
    return train_missing.sort_values(ascending = False)

def get_date_cols(df = None):
    date_cols =[]
    for col in df.columns:
        if 'Date' in col:
            date_cols.append(col)
    if len(date_cols) > 0:
        return date_cols
    else:
        return('The COlumn name doesnot have Date string')



#Date Conversion:
def date_format_conversion(df = None):
    """
    Function converts all the columns having 'Date' in datetimestamp
    """
    date_cols = get_date_cols(df)
    for col in df.columns:
        if col in date_cols:
            df[col] = pd.to_datetime(df[col])
    return df


def valid_age(age:int):
    if age < 0:
        if abs(age) >= 18 and abs(age) < 100:
            return int(abs(age))
    elif age < 18:
        return np.nan
    elif age > 100:
        return np.nan
    else:
        return int(age)


def gender_map(gender:object):

    if type(gender) != str:
        return 'other'
    else:
        if gender.lower() == 'male':
            return 'male'
        elif gender.lower() == 'female':
            return 'female'
        else:
            return 'other'

def generate_policy_tenure(df,granularity = "months"):
    policy_tenure_days = (df['Policy End Date'] - df['Policy Start Date']).dt.days
     
    if granularity.lower() == "months":
        df["Policy_Tenure"+"_"+granularity] = round(policy_tenure_days/30)
    

    return df
        

     
    

def create_date_features(df,colname):
    '''
        Creates date features like month,day, year etc.

        Feel free to add more features and customize
    '''

    df[colname+'_month'] = df[colname].dt.month
    df[colname+'_year'] = df[colname].dt.year
    df[colname+'_day_of_week'] = df[colname].dt.dayofweek

    df= df.drop(colname,axis=1)

    return df

## Reading and Cleaning the data:


In [36]:
raw_df = pd.read_csv('Data/Train.csv')

In [37]:
df_null_percentage(raw_df)

Subject_Car_Colour        57.637222
State                     53.713056
LGA_Name                  53.613710
Car_Category              30.946270
Subject_Car_Make          20.498386
Gender                     2.972100
ID                         0.000000
Policy Start Date          0.000000
Policy End Date            0.000000
Age                        0.000000
First Transaction Date     0.000000
No_Pol                     0.000000
ProductName                0.000000
target                     0.000000
dtype: float64

In [38]:
# Sanity Checks for Age and Gender
raw_df.Gender= raw_df.Gender.map(lambda x: gender_map(x))
raw_df.Age = raw_df.Age.map(lambda x: valid_age(x))

In [39]:
categorical_features = list(raw_df.select_dtypes(['O']).nunique().index)

In [40]:
date_cols = get_date_cols(raw_df)

In [41]:


categorical_features= set(categorical_features) - set(date_cols) 
categorical_features.remove("ID")
categorical_features = list(categorical_features)
print(categorical_features)


['Gender', 'ProductName', 'Car_Category', 'Subject_Car_Make', 'LGA_Name', 'State', 'Subject_Car_Colour']


In [42]:
# In this iteration, Will Impute the values for : Subject_Car_Colour, State ,LGA_Name 

# Cell to Impute all Categorical Missing values with the Modes


for column in categorical_features:

    raw_df[column].fillna(raw_df[column].mode()[0], inplace=True)



In [43]:
# Fixing the missing values for Continous/ Numerical Columns
numerical_features = list(raw_df.select_dtypes(['int64','float64']).nunique().index)
for column in numerical_features:

    raw_df[column].fillna(round(raw_df[column].mean()), inplace=True)

In [44]:
insurance_application_df = date_format_conversion(raw_df)

In [45]:
df_null_percentage(insurance_application_df)


ID                        0.0
Policy Start Date         0.0
Policy End Date           0.0
Gender                    0.0
Age                       0.0
First Transaction Date    0.0
No_Pol                    0.0
Car_Category              0.0
Subject_Car_Colour        0.0
Subject_Car_Make          0.0
LGA_Name                  0.0
State                     0.0
ProductName               0.0
target                    0.0
dtype: float64

In [46]:
print(f"Shape of the cleaned data: {insurance_application_df.shape}")

Shape of the cleaned data: (12079, 14)


In [47]:
#We can see that the Policy Start Date Date is as same as First Transaction Date, we can drop one from our analysis
#sum(insurance_application_df["Policy Start Date"] != insurance_application_df["First Transaction Date"])
insurance_application_df.drop(["First Transaction Date"],1, inplace=True)
print(f"{get_date_cols(insurance_application_df)}")

['Policy Start Date', 'Policy End Date']


## Feature Creation

 Policy Tenure creation

We can see that we have three date columns given-

* Policy Start Date
* Policy End Date
* First Transaction Date

We can create some meaningful features that will answer the questions like below- 
* What was the tenure of policy in months? 
* What were the months, days, etc corresponding to a given Policy Start and End date. 
* If First Transaction date is similar to Policy Start Date, can we drop it ? 

In [48]:
#Policy Tenure in Months:
insurance_application_df = generate_policy_tenure(insurance_application_df)


In [49]:
date_features_final = get_date_cols(insurance_application_df)
for date_col in date_features_final :
   insurance_application_df = create_date_features(insurance_application_df,date_col)
    


In [50]:
# Age should be an int value
insurance_application_df.Age = insurance_application_df.Age.map(lambda x: int(x))

In [53]:
# Policy Tenure should be int value
insurance_application_df.Policy_Tenure_months = insurance_application_df.Policy_Tenure_months.map(lambda x: int(x))

In [56]:
insurance_application_df.head()

,ID,Gender,Age,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target,Policy_Tenure_months,Policy Start Date_month,Policy Start Date_year,Policy Start Date_day_of_week,Policy End Date_month,Policy End Date_year,Policy End Date_day_of_week
0,ID_0040R73,male,30,1,Saloon,Black,TOYOTA,Victoria Island,Lagos,Car Classic,0,12,5,2010,4,5,2011,4
1,ID_0046BNK,female,79,1,JEEP,Grey,TOYOTA,Victoria Island,Lagos,Car Classic,1,12,11,2010,0,11,2011,0
2,ID_005QMC3,male,43,1,Saloon,Red,TOYOTA,Victoria Island,Lagos,Car Classic,0,12,3,2010,6,3,2011,6
3,ID_0079OHW,male,43,1,Saloon,Black,TOYOTA,Victoria Island,Lagos,CarSafe,0,12,8,2010,5,8,2011,5
4,ID_00BRP63,other,20,3,Saloon,Black,TOYOTA,Lagos,Lagos,Muuve,1,4,8,2010,6,12,2010,4
